<a href="https://colab.research.google.com/github/vishalvv99/NLP/blob/main/project_on_named_entity_recognitation_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U spacy
!python -m spacy download en_core_web_lg
!python -m spacy download en_core_web_trf

In [ ]:
import spacy
from spacy.training.example import Example

In [ ]:
# Load the English language model in spaCy
nlp = spacy.load("en_core_web_lg")

In [ ]:
# Sample text for NER
text = """As of my last knowledge update in January 2022, I don't have specific information about a company named Boat India.
However, I can provide information about boAt Lifestyle, an Indian consumer electronics and audio accessories brand that was well-known at that time.
Please note that there might have been developments or changes since then. boAt Lifestyle is known for manufacturing and selling a wide range of audio products,
including headphones, earphones, speakers, and other accessories.
The company gained popularity for offering affordable yet stylish audio devices, catering to the demands of the youth and tech-savvy consumers in India.
They often collaborated with celebrities and influencers for product endorsements and marketing. For the most up-to-date information, it is recommended to check the latest news or visit the official website of
boAt Lifestyle or any other specific entity you are referring to as Boat India.
World War II (WWII) was a global conflict that lasted from 1939 to 1945, involving the majority of the world's nations.
It was the deadliest and most widespread war in human history, with key contributing factors including economic instability, political unrest, and unresolved tensions from World War I."""

In [ ]:
# Process the text with the NLP pipeline
doc = nlp(text)

In [ ]:
# Iterate through entities identified in the text and print them
for ent in doc.ents:
    print(ent.text, ent.label_)

In [ ]:
doc.ents

In [ ]:
type(doc)

In [ ]:
doc.ents[5], type(doc.ents[0])

In [ ]:

from spacy import displacy
displacy.render(doc, style="ent", jupyter = True)

### CREATING NEW NER SPACY

In [ ]:
import json #here we are using json file
from google.colab import drive
drive.mount('/content/drive')

#https://www.kaggle.com/datasets/finalepoch/medical-ner
path = '/content/drive/MyDrive/NLP/Corona2.json'

with open(path, 'r') as file:
    data = json.load(file)

In [ ]:
# Display the content
print(data)

In [ ]:
data['examples']

In [ ]:
data['examples'][0]

In [ ]:
data['examples'][0].keys()

In [ ]:
data['examples'][0]['content']

In [ ]:
data['examples'][0]['annotations'][0]

In [ ]:
train_data =[]
for example in data['examples']:
  temp_dict ={}
  temp_dict['text'] = example ['content']
  temp_dict['entities']= []
  for annotation in example ['annotations']:
    start = annotation ['start']
    end = annotation ['end']
    label = annotation ['tag_name'].upper()
    temp_dict['entities'].append((start,end, label))
  train_data.append(temp_dict)

print(train_data[0])

In [ ]:
train_data[0]['text']

In [ ]:
train_data[0]['entities']

In [ ]:
train_data[0]['text'][870 : 880]

In [ ]:
from spacy.tokens import DocBin
from tqdm import tqdm

#to load new spacy model
nlp = spacy.blank("en")
doc_bin =DocBin()

In [ ]:
from spacy.util import filter_spans

for train_example in tqdm(train_data):
    text = train_example['text']
    labels = train_example['entities']
    doc= nlp.make_doc(text)
    ents=[]
    for start, end, label in labels:
      span = doc.char_span(start, end, label= label, alignment_mode = "contract")
      if span is None:
        print("Skipping Entity")
      else:
        ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)

doc_bin.to_disk("train.spacy")

In [ ]:
#https://spacy.io/usage/training#quickstart


!python -m spacy init fill-config base_config.cfg config.cfg


In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

In [ ]:
nlp_ner = spacy.load("model-best")

In [ ]:
doc = nlp_ner(text)

doc

In [ ]:
colors = {"PATHOGEN": "#90EE90", "MEDICINE" : "#FF00FF", "MEDICALCONDITION" : "#00FFFF"}
options = {"colors" : colors}

spacy.displacy.render(doc, style="ent", options = options, jupyter = True)